In [ ]:
# --------------------------------------------------
# 0. Instalación y Carga de Paquetes
# --------------------------------------------------

# install.packages(c("readr", "dplyr", "hdm", "glmnet", "randomForest", "sandwich", "nnet"))

library(readr)       # Para leer el archivo .dat
library(dplyr)       # Para manipulación de datos (filter, mutate)
library(hdm)         # Para rlasso (como en tu lab)
library(glmnet)      # Para OLS, Logit y Lasso Logit
library(randomForest)# Para Random Forest
library(sandwich)    # Para SE robustos (vcovHC)
library(nnet)        # Para Redes Neuronales

set.seed(123)

# --------------------------------------------------
# 1. Carga y Limpieza de Datos (Parte I)
# --------------------------------------------------

# Nombres de las columnas del archivo .dat
nombres <- c(
    "abdt", "tg", "inuidur1", "inuidur2", "female", "black", "hispanic", 
    "othrace", "dep", "q1", "q2", "q3", "q4", "q5", "q6", "recall", 
    "agelt35", "agegt54", "durable", "nondurable", "lusd", "husd", "muld"
)

# Descargar y leer el archivo de datos
url <- "https://raw.githubusercontent.com/CausalAIBook/MetricsMLNotebooks/main/data/penn_jae.dat"
df <- read_delim(url, delim = " ", col_names = nombres, skip = 1, trim_ws = TRUE)

# Aplicar la limpieza y creación de variables de tu notebook de Python
df_cleaned <- df %>%
  filter(tg == 0 | tg == 4) %>%
  mutate(
    # II. Definir 'T4' (d, tratamiento)
    T4 = as.integer(tg == 4),
    
    # III. Definir 'y' (resultado)
    y = log(inuidur1),
    
    # IV. Crear dummies para 'dep'
    dep_1 = as.integer(dep == 1),
    dep_2 = as.integer(dep == 2)
  ) %>%
  # Manejar log(0) = -Inf, similar a como lo hiciste en Python
  filter(is.finite(y))

print(paste("Datos después de limpiar:", nrow(df_cleaned)))

# V. Definir x (controles), y (resultado), d (tratamiento)
feature_list <- c(
    "female", "black", "othrace",
    "dep_1", "dep_2",
    "q2", "q3", "q4", "q5", "q6",
    "recall", "agelt35", "agegt54",
    "durable", "nondurable", "lusd", "husd"
)

x <- as.matrix(df_cleaned[, feature_list])
y <- as.matrix(df_cleaned$y)
d <- as.matrix(df_cleaned$T4)


# --------------------------------------------------
# 2. Funciones DML (Parte II y III)
# --------------------------------------------------

# ---
# Función DML con Cross-Fitting (Parte II)
# Basada en tu lab, pero modificada para manejar
# los diferentes 'tipos' de predicción (ej. 'prob' para RF, 'response' para glmnet)
# ---
DML.for.PLM <- function(x, d, y, dreg, yreg, nfold=10, d_pred_type="response", y_pred_type="response") {
  nobs <- nrow(x)
  # Crear pliegues (folds)
  foldid <- rep.int(1:nfold, times = ceiling(nobs/nfold))[sample.int(nobs)]
  I <- split(1:nobs, foldid)
  ytil <- dtil <- rep(NA, nobs)
  
  cat("Fold: ")
  for(b in 1:length(I)){
    # Ajustar modelos en los pliegues de entrenamiento (todos menos 'b')
    dfit <- dreg(x[-I[[b]],], d[-I[[b]]])
    yfit <- yreg(x[-I[[b]],], y[-I[[b]]])
    
    # Predecir en el pliegue de prueba ('b')
    
    # Manejar la predicción de D (tratamiento)
    if (d_pred_type == "prob") {
        # Para randomForest, 'prob' da 2 columnas (prob 0, prob 1). Queremos la segunda.
        dhat <- predict(dfit, x[I[[b]],], type="prob")[, 2]
    } else if (d_pred_type == "raw" && inherits(dfit, "nnet")) {
        # Para nnet, 'raw' da probabilidades. Si es binario, puede dar 1 o 2 columnas.
        pred_raw <- predict(dfit, x[I[[b]],], type="raw")
        dhat <- if (ncol(pred_raw) == 2) pred_raw[, 2] else pred_raw[, 1]
    } else {
        # Para glmnet (logit y lasso), 'response' da la probabilidad P(D=1)
        dhat <- predict(dfit, x[I[[b]],], type=d_pred_type)
    }
    
    # Manejar la predicción de Y (resultado)
    yhat <- predict(yfit, x[I[[b]],], type=y_pred_type)
    
    # Calcular residuales
    dtil[I[[b]]] <- (d[I[[b]]] - dhat)
    ytil[I[[b]]] <- (y[I[[b]]] - yhat)
    cat(b, " ")
  }
  
  # Regresión final de residuales
  rfit <- lm(ytil ~ dtil)
  coef.est <- coef(rfit)[2]
  se <- sqrt(vcovHC(rfit)[2,2]) # Error estándar robusto
  
  cat(sprintf("\nCoef (SE) = %g (%g)\n", coef.est, se))
  return( list(coef.est=coef.est , se=se, dtil=dtil, ytil=ytil) )
}

# ---
# Función DML Naive (Sin Cross-Fitting) (Parte III)
# ---
DML.naive <- function(x, d, y, dreg, yreg, d_pred_type="response", y_pred_type="response") {
  
  # 1. Entrenar y predecir Y (en muestra)
  yfit <- yreg(x, y)
  yhat <- predict(yfit, x, type=y_pred_type)
  
  # 2. Entrenar y predecir D (en muestra)
  dfit <- dreg(x, d)
  
  # Manejar la predicción de D (tratamiento)
  if (d_pred_type == "prob") {
      dhat <- predict(dfit, x, type="prob")[, 2]
  } else if (d_pred_type == "raw" && inherits(dfit, "nnet")) {
      pred_raw <- predict(dfit, x, type="raw")
      dhat <- if (ncol(pred_raw) == 2) pred_raw[, 2] else pred_raw[, 1]
  } else {
      dhat <- predict(dfit, x, type=d_pred_type)
  }

  # 3. Calcular residuales
  dtil <- (d - dhat)
  ytil <- (y - yhat)
  
  # 4. Regresión final
  rfit <- lm(ytil ~ dtil)
  coef.est <- coef(rfit)[2]
  se <- sqrt(vcovHC(rfit)[2,2])
  
  cat(sprintf("\nCoef (SE) = %g (%g)\n", coef.est, se))
  return( list(coef.est=coef.est , se=se, dtil=dtil, ytil=ytil) )
}


# --------------------------------------------------
# 3. Parte II: Ejecutar DML (con Cross-Fitting)
# --------------------------------------------------

# --- 3.1 Definir los wrappers de los modelos ---

# 1. OLS/Logit
# glmnet con lambda=0 es OLS
yreg_ols <- function(x, y) glmnet(x, y, lambda = 0)
# glmnet con family="binomial" es Logit
dreg_ols <- function(x, d) glmnet(x, d, family = "binomial", lambda = 0)

# 2. Lasso
yreg_lasso <- function(x, y) rlasso(x, y, post=FALSE) # De hdm
# cv.glmnet con alpha=1 es Lasso, CV selecciona lambda.
dreg_lasso <- function(x, d) cv.glmnet(x, d, family = "binomial", alpha = 1)

# 3. Random Forest
yreg_rf <- function(x, y) randomForest(x, y) #
# d debe ser un factor para que randomForest haga clasificación
dreg_rf <- function(x, d) randomForest(x, as.factor(d))

# 4. Neural Net (usando nnet)
# 'linout=TRUE' es para regresión. 'size' es el nro. de nodos en la capa oculta.
# Aumentamos MaxNWts para permitir más pesos (features -> nodos)
yreg_nn <- function(x, y) nnet(x, y, size=20, linout=TRUE, trace=FALSE, MaxNWts=2000)
dreg_nn <- function(x, d) nnet(x, as.factor(d), size=20, trace=FALSE, MaxNWts=2000)


# --- 3.2 Ejecución ---
cat("\n--- DML con OLS/Logit ---\n")
res_ols <- DML.for.PLM(x, d, y, dreg_ols, yreg_ols, nfold=10, 
                       d_pred_type="response", y_pred_type="response")

cat("\n--- DML con Lasso ---\n")
res_lasso <- DML.for.PLM(x, d, y, dreg_lasso, yreg_lasso, nfold=10, 
                         d_pred_type="response", y_pred_type="response")

cat("\n--- DML con Random Forest ---\n")
res_rf <- DML.for.PLM(x, d, y, dreg_rf, yreg_rf, nfold=10, 
                      d_pred_type="prob", y_pred_type="response")

cat("\n--- DML con Neural Net ---\n")
res_nn <- DML.for.PLM(x, d, y, dreg_nn, yreg_nn, nfold=10, 
                      d_pred_type="raw", y_pred_type="raw")

# --- 3.3 Crear Tabla de Resultados (Parte II) ---
table_dml <- data.frame(
  Estimate = c(res_ols$coef.est, res_lasso$coef.est, res_rf$coef.est, res_nn$coef.est),
  `Std. Error` = c(res_ols$se, res_lasso$se, res_rf$se, res_nn$se),
  `RMSE Y` = c(sqrt(mean(res_ols$ytil^2)), sqrt(mean(res_lasso$ytil^2)), sqrt(mean(res_rf$ytil^2)), sqrt(mean(res_nn$ytil^2))),
  `RMSE D` = c(sqrt(mean(res_ols$dtil^2)), sqrt(mean(res_lasso$dtil^2)), sqrt(mean(res_rf$dtil^2)), sqrt(mean(res_nn$dtil^2))),
  row.names = c("OLS/Logit", "Lasso", "Random Forest", "NN (nnet)"),
  check.names = FALSE
)

cat("\n--- Resultados DML (Cross-Fit) ---\n")
print(table_dml, digits=4)


# --------------------------------------------------
# 4. Parte III: Ejecutar DML Naive (Sin Cross-Fitting)
# --------------------------------------------------

cat("\n\n--- Naive DML con OLS/Logit ---\n")
res_ols_n <- DML.naive(x, d, y, dreg_ols, yreg_ols, 
                       d_pred_type="response", y_pred_type="response")

cat("\n--- Naive DML con Lasso ---\n")
res_lasso_n <- DML.naive(x, d, y, dreg_lasso, yreg_lasso, 
                         d_pred_type="response", y_pred_type="response")

cat("\n--- Naive DML con Random Forest ---\n")
res_rf_n <- DML.naive(x, d, y, dreg_rf, yreg_rf, 
                      d_pred_type="prob", y_pred_type="response")

cat("\n--- Naive DML con Neural Net ---\n")
res_nn_n <- DML.naive(x, d, y, dreg_nn, yreg_nn, 
                      d_pred_type="raw", y_pred_type="raw")

# --- 4.1 Crear Tabla de Resultados (Parte III) ---
table_naive <- data.frame(
  Estimate = c(res_ols_n$coef.est, res_lasso_n$coef.est, res_rf_n$coef.est, res_nn_n$coef.est),
  `Std. Error` = c(res_ols_n$se, res_lasso_n$se, res_rf_n$se, res_nn_n$se),
  `RMSE Y` = c(sqrt(mean(res_ols_n$ytil^2)), sqrt(mean(res_lasso_n$ytil^2)), sqrt(mean(res_rf_n$ytil^2)), sqrt(mean(res_nn_n$ytil^2))),
  `RMSE D` = c(sqrt(mean(res_ols_n$dtil^2)), sqrt(mean(res_lasso_n$dtil^2)), sqrt(mean(res_rf_n$dtil^2)), sqrt(mean(res_nn_n$dtil^2))),
  row.names = c("OLS/Logit", "Lasso", "Random Forest", "NN (nnet)"),
  check.names = FALSE
)

cat("\n--- Resultados Naive DML (Sin Cross-Fit) ---\n")
print(table_naive, digits=4)


# --------------------------------------------------
# 5. Comparación y Preguntas
# --------------------------------------------------

# Añadir columna de método
table_dml$Method <- "DML (Cross-Fit)"
table_naive$Method <- "Naive (No Cross-Fit)"

# Combinar
comparison_table <- rbind(table_dml, table_naive)
comparison_table <- comparison_table[order(rownames(comparison_table)), ] # Ordenar por nombre de modelo

cat("\n\n--- Comparación Completa: DML vs. Naive ---\n")
print(comparison_table, digits=4)

# --- Respuestas a las Preguntas ---
cat("
### Pregunta 1: What can you say about the RMSE for predicting y and d?

Respuesta: Al observar la tabla de comparación, se ve que los RMSE de 'RMSE Y' y 'RMSE D'
son consistentemente más bajos para el método 'Naive (No Cross-Fit)' que para 'DML (Cross-Fit)'.
Esta diferencia es especialmente grande para los modelos más flexibles como Random Forest y NN.

### Pregunta 2: Why is it that estimating with one function yields lower RMSE than another?

Respuesta: Esto se debe al **sobreajuste (overfitting)**.
- La función **Naive** entrena y evalúa el modelo en los *mismos datos* (error en-muestra o 'in-sample').
  Los modelos flexibles (RF, NN) son excelentes para 'memorizar' los datos de entrenamiento,
  incluido el ruido, lo que resulta en un RMSE artificialmente bajo.
- La función **DML (Cross-Fit)** entrena en un subconjunto y evalúa en un conjunto *no visto*
  (error fuera-de-muestra o 'out-of-sample'). Esta es una medida más honesta del rendimiento
  real del modelo.

### Pregunta 3: What problem would we have if we chose to estimate without cross-fitting?

Respuesta: El problema principal es el **sesgo por sobreajuste (overfitting bias)**.

La teoría de Double Machine Learning (DML) requiere que los residuos (ytil, dtil) se
generen de forma 'ortogonal' al proceso de estimación. El cross-fitting es el
mecanismo que lo permite.

Si *no* usamos cross-fitting (el método Naive):
1. Los modelos se sobreajustan a los datos.
2. Los residuos `ytil` y `dtil` están 'contaminados' por este sobreajuste.
3. La regresión final de `ytil ~ dtil` estará **sesgada**, y el estimador
   del efecto causal no será válido ni confiable.

Básicamente, sin cross-fitting, sacrificamos la validez estadística por
una falsa sensación de precisión (un RMSE más bajo).
")